In [1]:
from docplex.mp.model import Model
import time
import random
import pandas as pd

In [2]:
# Define the maximization of net profit problem
opt_mod = Model(name = "Linear Program")

In [3]:
# parameters to be defined before starting the problem
n=150000

p1 = 4.0 # Customer profit per item lower bound
p2 = 6.0 # Customer profit per item upper bound
p3 = 0.75 # Customer penalty per item lower bound
p4 = 1.75 # Customer penalty per item upper bound
m1 = 500 # Maximum demand lower bound
m2 = 2000 # Maximum demand upper bound
m3 = 10 # Minimum demand

a = 1.25     #Average square foot space required to sort 1 package
b = 300*n    #Max available square foot
d = b/a      #Maximum number of items to be sorted for packages

In [4]:
# Create Random n datapoints

starttime = time.time()

cusnum =[]
profits=[]
penalties=[]
maximums=[]
minimums=[]

for i in range(1,n+1):
    cusnum.append(i)
    profits.append(round(random.uniform(p1,p2),2))
    penalties.append(round(random.uniform(p3,p4),2))
    maximums.append(random.randint(m1,m2))
    minimums.append(10)

df = pd.DataFrame(columns=['Customer Number', 'Profit', 'Penalty','Minimum','Maximum'])

df['Customer Number']=cusnum
df['Profit']=profits
df['Penalty']=penalties
df['Maximum']=maximums
df['Minimum']=minimums


In [5]:
# Create a list of the customer numbers
customer_numbers = list(df['Customer Number'])

# Create a list of profits for all customers
Profits = list(df['Profit'])

# Create a list of penalties for all customers
Penalties = list(df['Penalty'])

# Create a list of minimums for all customers
Minimums = list(df['Minimum'])

# Create a list of maximums for all customers
Maximums = list(df['Maximum'])

In [6]:
# Assigning the decision variables

In [7]:
# Variable x represents a list of the amount to be processed for each customer
x=opt_mod.integer_var_list(n , name= "x")

In [8]:
### Assigning the Constraints

In [9]:
# Total square footage has to be less than maximum of b, maximum sqft available
c1 = opt_mod.add_constraint( sum(a*x[i] for i in range(n))<=b, ctname = 'sqftmaximum')

# Total processed for customers has to be less than d, maximum allowable processing
c2 = opt_mod.add_constraint( sum(x[i] for i in range(n))<=d, ctname = 'processmaximumtotal')


for f in range(n):
    # Processed for one customer has to be less than the maximum for that customer
    opt_mod.add_constraint(x[f]<=Maximums[f])
    # Processed for one customer has to be less than the minimum for that customer
    opt_mod.add_constraint(x[f]>=Minimums[f])

In [10]:
### Write the objective function to optimize

In [11]:
# Building objective function to maximize net profit
obj_fn=sum(Profits[i]*x[i]-Penalties[i]*(Maximums[i]-x[i]) for i in range(n))
opt_mod.set_objective('max',obj_fn)

In [12]:
### View Details of the model
opt_mod.print_information()

Model: Linear Program
 - number of variables: 150000
   - binary=0, integer=150000, continuous=0
 - number of constraints: 300002
   - linear=300002
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [13]:
#Assign which method will be used to solve
opt_mod.parameters.lpmethod=2

#0 CPX_ALG_AUTOMATIC Automatic: let CPLEX choose; default
#1 CPX_ALG_PRIMAL Primal simplex
#2 CPX_ALG_DUAL Dual simplex
#3 CPX_ALG_NET Network simplex
#4 CPX_ALG_BARRIER Barrier
#5 CPX_ALG_SIFTING Sifting
#6 CPX_ALG_CONCURRENT Concurrent (Dual, Barrier, and Primal in opportunistic parallel mode; Dual and Barrier in deterministic parallel mode)

In [14]:
opt_mod.solve(log_output=True)
print('Optimization done, The maximum Revenue is :$%.2f' %opt_mod.objective_value)

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                2
Found incumbent of value -2.2464220e+08 after 0.03 sec. (16.49 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 300002 rows and 150000 columns.
MIP Presolve modified 1 coefficients.
All rows and columns eliminated.
Presolve time = 0.20 sec. (192.63 ticks)

Root node processing (before b&c):
  Real time             =    0.23 sec. (215.98 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.23 sec. (215.98 ticks)
Optimization done, The maximum Revenue is :$22884407.09


In [15]:
#opt_mod.print_solution()

In [16]:
#Save the results to a dataframe
results = pd.DataFrame(columns=['Customer Number', 'Optimal number to process', 'Square footage utilized'])
results['Optimal number to process']=opt_mod.solution.get_value_list(x)
results['Customer Number']=cusnum
results['Square footage utilized']= a*results['Optimal number to process']

In [17]:
# Merge input data into results dataframe
results = pd.merge(results, df, on ='Customer Number', how ='inner')

In [18]:
results

,Customer Number,Optimal number to process,Square footage utilized,Profit,Penalty,Minimum,Maximum
0,1,10.0,12.50,5.39,1.47,10,551
1,2,10.0,12.50,4.47,1.73,10,556
2,3,576.0,720.00,5.83,1.40,10,576
3,4,10.0,12.50,5.52,1.17,10,1163
4,5,1405.0,1756.25,5.93,1.55,10,1405
...,...,...,...,...,...,...,...
149995,149996,10.0,12.50,5.52,1.31,10,974
149996,149997,10.0,12.50,5.11,1.71,10,849
149997,149998,10.0,12.50,5.14,1.57,10,1442
149998,149999,10.0,12.50,4.13,1.07,10,826


In [19]:
# Create columns for total profit, total penalty, net profit
results["Total Profit"]=results["Profit"]*results["Optimal number to process"]
results["Total Penalty"]=results["Penalty"]*(results["Maximum"]-results["Optimal number to process"])
results["Net Profit"]=results["Total Profit"]-results["Total Penalty"]

In [20]:
results

,Customer Number,Optimal number to process,Square footage utilized,Profit,Penalty,Minimum,Maximum,Total Profit,Total Penalty,Net Profit
0,1,10.0,12.50,5.39,1.47,10,551,53.90,795.27,-741.37
1,2,10.0,12.50,4.47,1.73,10,556,44.70,944.58,-899.88
2,3,576.0,720.00,5.83,1.40,10,576,3358.08,0.00,3358.08
3,4,10.0,12.50,5.52,1.17,10,1163,55.20,1349.01,-1293.81
4,5,1405.0,1756.25,5.93,1.55,10,1405,8331.65,0.00,8331.65
...,...,...,...,...,...,...,...,...,...,...
149995,149996,10.0,12.50,5.52,1.31,10,974,55.20,1262.84,-1207.64
149996,149997,10.0,12.50,5.11,1.71,10,849,51.10,1434.69,-1383.59
149997,149998,10.0,12.50,5.14,1.57,10,1442,51.40,2248.24,-2196.84
149998,149999,10.0,12.50,4.13,1.07,10,826,41.30,873.12,-831.82


In [24]:
# Save results to excel csv
results.to_csv('CPLEXresults150k.csv',index=False)

In [22]:
endtime = time.time()
print("--- %s seconds ---" % (endtime - starttime))

--- 1187.7321407794952 seconds ---


In [23]:
print('Optimization done, The maximum Revenue is :$%.2f' %opt_mod.objective_value)

Optimization done, The maximum Revenue is :$22884407.09
